In [1]:
from eunjeon import Mecab
import eunjeon
import konlpy

# numpy & pandas
import numpy as np
import pandas as pd

# scikit learn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Layer, Dense, Embedding, Activation, LSTM, Bidirectional, GRU, Softmax, Dropout
from keras.preprocessing.sequence import pad_sequences

# my lib
import textlib as tl
import Word2VecModel as wv

In [2]:
w2v_model_file_name_prefix = f'D:/dataset/w2v/telco_w2v_'

In [3]:
# 여러개의 w2v 모델을 만들기 위한 table
MODEL_COUNT = 9
W2V_TRAIN_PARAMS = {
    'MODEL_NAME': ['V10000_E100_W3_E50','V10000_E100_W5_E50','V10000_E100_W7_E50',
                   'V10000_E200_W3_E50','V10000_E200_W5_E50','V10000_E200_W7_E50',
                   'V10000_E300_W3_E50','V10000_E300_W5_E50','V10000_E300_W7_E50'],
    'MAX_VOCAB_SIZE': [10000] * MODEL_COUNT,
    'EMBEDDING_SIZE': [100,100,100,200,200,200,300,300,300],
    'WINDOW_SIZE' : [3,5,7,3,5,7,3,5,7],
    'EPOCHS': [40] * MODEL_COUNT
}

# parameter 잘못 넣었는지 검증
assert len(W2V_TRAIN_PARAMS['MODEL_NAME']) == MODEL_COUNT
assert len(W2V_TRAIN_PARAMS['MAX_VOCAB_SIZE']) == MODEL_COUNT
assert len(W2V_TRAIN_PARAMS['EMBEDDING_SIZE']) == MODEL_COUNT
assert len(W2V_TRAIN_PARAMS['WINDOW_SIZE']) == MODEL_COUNT
assert len(W2V_TRAIN_PARAMS['EPOCHS']) == MODEL_COUNT

# 여기까지 word2vec 모델 만들어서 파일 쓰는 것까지!!!(upstream task)
# 이 밑에서부터는 SOR 을 분류하는 downstream task!!

In [4]:
# loss, accuracy plot
def plot_hist(hist):
    import matplotlib.pyplot as plt

    fig, axes = plt.subplots(1,2)
    loss_ax = axes[0]
    acc_ax = axes[1]

    loss_ax.plot(hist.history['loss'], 'y', label='train loss')
    loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    loss_ax.legend(loc='upper right')

    acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
    acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')
    acc_ax.set_ylabel('accuracy')
    acc_ax.legend(loc='lower right')

    plt.show()    

In [5]:
# sor dataset 읽어옴
input_file_name = 'D:/dataset/sor_dataset.xlsx'
try:
    df = pd.read_excel(input_file_name, sheet_name=0, engine='openpyxl')
except FileNotFoundError:
    print(f'{input_file_name}이 없습니다! skip!')

print( df.shape )


(105268, 5)


In [6]:
# null 인 row가 하나라도 있으면 삭제
df.dropna(axis=0, inplace=True)
print( df.shape )

(92483, 5)


In [7]:
# co(요청회사)가 SKT, SKB인 것만 추출. 너무 적은 표본도 삭제
df = df[ ((df['co'] == 'SKT') | (df['co'] == 'SKB')) & \
             (
                (df['label'] != 'Configuration') & 
                (df['label'] != 'EAI/EIGW') &
                (df['label'] != 'I/F 유틸') &
                (df['label'] != 'MTOKTOK') &
                (df['label'] != 'PPS 상품권') &
                (df['label'] != 'Utility') &
                (df['label'] != '고객상담') &
                (df['label'] != '접근 관리') &
                (df['label'] != '코드 관리')
             )
       ]

print( df.shape )

(92197, 5)


In [8]:
# dataset의 문장 길이 통계
def avg(it):
    return sum(it) / len(it)

lens = [len(s) for s in df['sentence'].values]
#lens = [len(s) for s in X]

print( min(lens) )
print( max(lens) )
print( avg(lens) )

93
3522
384.6967471826632


In [9]:
# label 통계
df.groupby('label').count()

,req_ym,co,req_br,sentence
label,,,,
Billing/OSS,6699,6699,6699,6699
CC,6484,6484,6484,6484
CRM,24176,24176,24176,24176
CTC,1676,1676,1676,1676
Customer Care,11064,11064,11064,11064
ERP,5661,5661,5661,5661
OCEAN,35,35,35,35
OSS,5458,5458,5458,5458
PRM,2597,2597,2597,2597


In [10]:
# 첫 모델은 sentence와 label만 써보자
# df_zip = df[ ['sentence', 'label'] ]

y = df.pop('label')
X = df.pop('sentence')

In [11]:
# 문자열로 되어 있는 label을 categorical value로 변환
label_encoder = preprocessing.LabelEncoder()
y = label_encoder.fit_transform(y)

In [12]:
# 잘 변환됐나?
print( y[0:20] )
print( label_encoder.inverse_transform(y[0:20]) )

[ 8  8  8  8  4  3  1 15 18 15  4  2 15 15 15 15  4 15 23  8]
['PRM' 'PRM' 'PRM' 'PRM' 'Customer Care' 'CTC' 'CC' '기타' '빌링' '기타'
 'Customer Care' 'CRM' '기타' '기타' '기타' '기타' 'Customer Care' '기타' '영업기타'
 'PRM']


In [13]:
w2v_model = wv.Word2VecModel()
w2v_model.load(w2v_model_file_name_prefix + W2V_TRAIN_PARAMS['MODEL_NAME'][0])

In [14]:
# 전처리 테스트
print('--- raw sentence ---')
print( X[0] )
print()

print('--- 클렌징된 sentence ---')
cleansed_text = tl.clean_text( X[0] )
print( cleansed_text )
print()

print('--- 구두점 단위로 잘라서 배열화 ---')
tokenized_sentence = tl.segment_sentences(cleansed_text)
print( tokenized_sentence )
print()

print('--- 형태소 ---')
# SOR이니까 변경/요청 이란 단어는 항상 들어가므로 삭제
ignore_words = ['변경','요청']
corpora = ' '.join(tl.get_corpora(tokenized_sentence, ignore_words)).split(' ')
print(corpora)
print()

sequence = w2v_model.get_words_indexes(corpora)
print('word id')
print(sequence)
print()

print(y[0])
print(label_encoder.inverse_transform([y[100]]))


--- raw sentence ---
소속영업장 변경 요청(SKB사내유치본점 -＞ 엘에스통신-채널고객팀) . SR-1705-0886;- 해당 서비스번호로 개통이 됐다고 하는데 스윙에서 서비스 번호로 검색이 되지 않아서 ;변경이 어렵습니다.;- 유통지원센터에서 청약 등록 시, 창리정보통신으로 등록했으나 스윙 이관후 확인 시 소속영업장이 금란텔레콤으로 되어 있어 변경 요청 함 . 요청유형:자료수정,요청유형상세:PRM,검토/승인자성명:홍도희,검토/승인자사번:1700,검토승인자기간:2017-05-31,요청내용:1. 변경 전 유통망 : SKB사내유치본점2. 변경 후 유통망 : 엘에스통신(E00901)3. 서비스번호 : 7276564018 또는 1670-84914. 고객 : 세종화재해상자동차손해사정(주)5. 담당 AM : 박성M6. 적용 시점 : 6월 지급 분부터 적용(5월 영업에 대한)

--- 클렌징된 sentence ---
소속영업장 변경 요청 SKB사내유치본점 엘에스통신 채널고객팀 . SR 해당 서비스번호로 개통이 됐다고 하는데 스윙에서 서비스 번호로 검색이 되지 않아서 변경이 어렵습니다. 유통지원센터에서 청약 등록 시 창리정보통신으로 등록했으나 스윙 이관후 확인 시 소속영업장이 금란텔레콤으로 되어 있어 변경 요청 함 . 요청유형 자료수정 요청유형상세 PRM 검토/승인자성명 홍도희 검토/승인자사번 검토승인자기간 요청내용 . 변경 전 유통망 SKB사내유치본점 . 변경 후 유통망 엘에스통신 E . 서비스번호 또는 . 고객 세종화재해상자동차손해사정 주 . 담당 AM 박성M . 적용 시점 월 지급 분부터 적용 월 영업에 대한 

--- 구두점 단위로 잘라서 배열화 ---
['소속영업장 변경 요청 SKB사내유치본점 엘에스통신 채널고객팀', 'SR 해당 서비스번호로 개통이 됐다고 하는데 스윙에서 서비스 번호로 검색이 되지 않아서 변경이 어렵습니다', '유통지원센터에서 청약 등록 시 창리정보통신으로 등록했으나 스윙 이관후 확인 시 소속영업장이 금란텔레콤으로 되어 있어 변경 요

In [15]:
# raw 문장을 전처리(쪼개고 형태소로 분리하고 단어id로 바꾸고 padding까지)
def preprocess_sentence(X, w2v_model, max_sentence_len=200, ignore_words=None):
    # 문장 전처리
    preprocessed_X = []
    print(f'{len(X)} 개의 데이터 존재 확인!')

    for i, text in enumerate(X):
        try:
            # 클렌징
            cleansed_text = tl.clean_text(text)
        except TypeError:
            print(f'      {i+1} 번째 데이터에 문제가 있어 skip!')
            continue

        # 문장으로 분리하여 배열로 리턴
        sentences = tl.segment_sentences(cleansed_text)
        # 문장 배열을 입릭으로 받아 형태소로 쪼갠 뒤, 다시 하나의 문자열로 변환하여 저장
        corpora = ' '.join(tl.get_corpora(sentences, ignore_words)).split(' ')
        # 단어들을 id로 변환
        sequence = w2v_model.get_words_indexes(corpora)
        
        preprocessed_X.append(sequence)

        if i % 5000 == 0 and i > 0:
            print(f'      {i} 번째 데이터 처리 완료!')

    print('데이터 전체 처리 완료!')
    # 최대 길이까지만 문장 사용. 짧은 문장은 뒤쪽에 PADDING token으로 채워서 리턴
    preprocessed_X = pad_sequences( preprocessed_X, maxlen=max_sentence_len, padding='post', value=w2v_model.word2index[PAD_TOKEN] )
    print('PADDING 완료!')    
    return preprocessed_X
    

In [ ]:
IGNORED_WORDS = ['변경','요청']
MAX_SENTENCE_LEN = 200
preprocessed_X = preprocess_sentence(X, w2v_model, MAX_SENTENCE_LEN, IGNORED_WORDS)

92197 개의 데이터 존재 확인!
      5000 번째 데이터 처리 완료!
      10000 번째 데이터 처리 완료!
      15000 번째 데이터 처리 완료!
      20000 번째 데이터 처리 완료!
      25000 번째 데이터 처리 완료!
      30000 번째 데이터 처리 완료!
      35000 번째 데이터 처리 완료!
      40000 번째 데이터 처리 완료!


In [ ]:
# train / test 분리
X_train, X_test, y_train, y_test = \
    train_test_split(preprocessed_X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
# 배열을 numpy 로 변경
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test  = np.array(X_test)
y_test  = np.array(y_test)

print( len(X_train) )
print( len(X_test) )
print( len(y_train) )
print( len(y_test) )

In [ ]:
# 잘 됐나?
index = 800
print( [w2v_model.index2word[i] for i in X_train[index]] )
print( X_train[index] )
print( y_train[index] )
print( label_encoder.inverse_transform([y_train[index]]) )

In [ ]:
# attention class. (ratsgo 샘플 약간 변형)
class Attention(Layer):
    def __init__(self, units):
        super(Attention, self).__init__()
        # (batch, )
        self.w1 = Dense(units, activation='tanh')
        self.w2 = Dense(1)    
        self.softmax_ = Softmax(axis=1)
        
    def call(self, x):
        # (batch, seq, embedding_dim*2) -> # (batch, seq, dec_units)
        x = self.w1(x)
        # (batch, seq, dec_units) -> # (batch, seq, 1)
        score = self.softmax_( self.w2(x) )

        return tf.squeeze( tf.matmul(tf.transpose(x, perm=[0, 2, 1]), score ), axis=-1 )

In [ ]:
# 차원 테스트
Attention(128)( np.zeros([64, 200, 128*2]) )

In [ ]:
# DL 모델. 
class SORClassifier(Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, 
                 batch_size, embedding_weights, apply_attention, train_embedding_layer, dropout, classes):
        super(SORClassifier, self).__init__()
        self.batch_size = batch_size
        self.dec_units = dec_units
        self.embedding = Embedding(
            input_dim=vocab_size,
            output_dim=embedding_dim,
            weights=[embedding_weights])
        self.lstm = Bidirectional( LSTM(self.dec_units, return_sequences=apply_attention) )
        self.fc1 = Dense(128, activation='relu')
        self.fc2 = Dense(classes, activation='softmax')
        self.do1 = Dropout(dropout)
        self.do2 = Dropout(dropout)
        
        self.attention = Attention(self.dec_units)
        self.embedding.trainable = train_embedding_layer
        self.apply_attention = apply_attention
        
    # 단어id 벡터 -> embedding layer -> dropout -> bilstm -> (opt)attention -> fc -> droupout -> fc(softmax)
    def call(self, x):
        # (batch, seq) -> (batch, seq, embedding_dim)        
        x = self.embedding(x)
        x = self.do1(x)
        # (batch, seq, embedding_dim) -> (batch, seq, embedding_dim*2)        
        x = self.lstm(x)
        
        # (batch, seq, embedding_dim*2) -> (batch, embedding_dim)        
        if self.apply_attention:
            x = self.attention(x)
        
        # (batch, embedding_dim) -> (batch, 128)    
        x = self.fc1(x)
        x = self.do2(x)
        x = self.fc2(x)
        
        return x

In [ ]:
def train_and_evaluate(vocab_size, embedding_dim, dec_units, epochs, batch_size, 
       weights, apply_attention, train_embedding_layer, dropout, classes):
    model = SORClassifier(
        vocab_size, 
        embedding_dim, 
        dec_units, 
        batch_size,
        weights,
        apply_attention,
        train_embedding_layer,
        dropout,
        classes
    )

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    history = model.fit(
        X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)
    
    test_score = model.evaluate(X_test, y_test, verbose=2)
    
    plot_hist(history)
    
    return history, test_score

In [ ]:
hyper_params = {
    'apply_attention': [True],
    'train_embedding_layer': [False],
    'dropout': [0.3],
    'weights': ['weight']#, 'norm_weight']
}

histories = dict()
test_scores = dict()

DEC_UNITS = 128
EPOCHS=10
BATCH_SIZE = 64

for a in hyper_params['apply_attention']:
    for te in hyper_params['train_embedding_layer']:
        for do in hyper_params['dropout']:
            for w in hyper_params['weights']:
                vocab_size, embedding_dim = w2v_model.weight.shape
                history, test_score = \
                    train_and_evaluate(vocab_size, 
                                       embedding_dim,
                                       DEC_UNITS,
                                       EPOCHS,
                                       BATCH_SIZE,
                                       w2v_model.weight,
                                       a,  # attention 적용할지?
                                       te, # embedding layer를 훈련시킬지, freezing시킬지?
                                       do, # dropout 비율
                                       len(label_encoder.classes_)) # class 개수
                model_name = f'{a}-{te}-{do}-{w}-{i}'
                histories[model_name] = history
                test_scores[model_name] = test_score

In [ ]:
test_scores
